In [6]:
import os 
from dotenv import load_dotenv
import pandas as pd 
import openai
import huggingface 
from enum import Enum 

Loading mental elf data form hugginface (use hf cli access token for this project in the .env file)

In [11]:
from datasets import load_dataset
   # Load the dataset
dataset = load_dataset("Amod/mental_health_counseling_conversations")

   # Access the train, validation, or test split if available
train_data = dataset['train']  # or use 'validation', 'test' if those splits exist

README.md:   0%|          | 0.00/2.82k [00:00<?, ?B/s]

c:\Users\tejas\AppData\Local\pypoetry\Cache\virtualenvs\mental-health-agent-XcAaYoBL-py3.11\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tejas\.cache\huggingface\hub\datasets--Amod--mental_health_counseling_conversations. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


combined_dataset.json:   0%|          | 0.00/4.79M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

Save to disk

In [17]:
#Take the loaded dataset object and save it into a the file called data 
train_data.save_to_disk('dataset')


Saving the dataset (0/1 shards):   0%|          | 0/3512 [00:00<?, ? examples/s]

# Defining the explicit role as natural language input

In [64]:
#Open AI API expects data in strcutured input format:
# json

# {
#  "messages": [
#    { "role": "<role_name_1>", "content": "<custom_content>" },
#    { "role": "<role_name_2>", "content": "<custom_content>" },
#    { "role": "<role_name_3>", "content": "<custom_content>" }
#  ]
# }


 ## Need enum classes for valid type inputs for role variable, and content

In [134]:
from enum import Enum
from typing import List, Dict

# 1. Define an Enum class for RoleType
class RoleType(Enum):
    USER = "user"
    SYSTEM = "system"
    ASSISTANT = "assistant"

# 2. Role class to represent a role and its associated content
class Role(object):
    def __init__(self, 
                 role_type:RoleType, 
                 content:str):
        """
        Role class to represent a role and its associated content for a message
        Args:
            role_type (RoleType): The type of role
            content (str): The content of the role
        """
        self.role = role_type.value
        self.content = content
        self.value = {'role': self.role, 'content':self.content } 


# 3. Message class to combine roles and their contents in a structured message
#  define the messsage class here
class Message(object):
    def __init__(self, user_content, system_content, assistant_content):
        self.user_role = Role(role_type=RoleType.USER , content=user_content)
        self.system_role = Role(role_type=RoleType.SYSTEM , content=system_content)
        self.assistant_role = Role(role_type=RoleType.ASSISTANT, content=assistant_content)
        self.message = {'messages':[self.system_role.value, self.user_role.value, self.assistant_role.value, ]}


In [135]:
context = train_data[152]['Context']
response = train_data[152]['Response']
system_content = "You serve as a supportive and honest psychology and psychotherapy assistant. Your main duty is to offer compassionate, understanding, and non-judgmental responses to users seeking emotional and psychological assistance. Respond with empathy and exhibit active listening skills. Your replies should convey that you comprehend the user’s emotions and worries. In cases where a user mentions thoughts of self-harm, suicide, or harm to others, prioritize their safety. Encourage them to seek immediate professional help and provide emergency contact details as needed. It’s important to note that you are not a licensed medical professional. Refrain from diagnosing or prescribing treatments. Instead, guide users to consult with a licensed therapist or medical expert for tailored advice. Never store or disclose any personal information shared by users. Uphold their privacy at all times. Avoid taking sides or expressing personal viewpoints. Your responsibility is to create a secure space for users to express themselves and reflect. Always aim to foster a supportive and understanding environment for users to share their emotions and concerns. Above all, prioritize their well-being and safety."
message_obj = Message(user_content=context, system_content=system_content, assistant_content = response )

# Validate message object
print(message_obj.message)

#Note: I wonder if this can be simplified using a pydantic class?

{'messages': [{'role': 'system', 'content': 'You serve as a supportive and honest psychology and psychotherapy assistant. Your main duty is to offer compassionate, understanding, and non-judgmental responses to users seeking emotional and psychological assistance. Respond with empathy and exhibit active listening skills. Your replies should convey that you comprehend the user’s emotions and worries. In cases where a user mentions thoughts of self-harm, suicide, or harm to others, prioritize their safety. Encourage them to seek immediate professional help and provide emergency contact details as needed. It’s important to note that you are not a licensed medical professional. Refrain from diagnosing or prescribing treatments. Instead, guide users to consult with a licensed therapist or medical expert for tailored advice. Never store or disclose any personal information shared by users. Uphold their privacy at all times. Avoid taking sides or expressing personal viewpoints. Your responsib

## Sampling 100 context:response pairs for fine-tuning LLM
#### Note: One could question why sample randomly? Wouldn't it be better to fine-tune using a properly defined cohort of conversations that pertain to certain classes of mental health problems? Maybe. It can be argued that most mental health problems and the appropriate response in natural language are highly corelated. 

In [136]:
# Sample 100 items from the 'train' split
import random
from loguru import logger
sampled_dataset = random.choices(train_data, k=300)
train_dataset = []

# Print the sampled data to verify
print(sampled_dataset[1])


{'Context': "I'm a guy. If I don't like girls, nor do I like guys, does that mean I'm gay?", 'Response': "It doesn't sound like you are finding yourself attracted to anyone. \xa0It could mean that you just haven't connected with anyone you find attractive, or that you are asexual - essentially not oriented toward anyone. \xa0I would suggest doing some reading on asexuality and see if it connects to how you feel!"}


In [137]:
for row in sampled_dataset:
    message_obj = Message(user_content=row['Context'], 
                          system_content=system_content, 
                          assistant_content=row['Response'])
    
    train_dataset.append(message_obj.message)

# Print a sample message to verify
print(train_dataset[1])

{'messages': [{'role': 'system', 'content': 'You serve as a supportive and honest psychology and psychotherapy assistant. Your main duty is to offer compassionate, understanding, and non-judgmental responses to users seeking emotional and psychological assistance. Respond with empathy and exhibit active listening skills. Your replies should convey that you comprehend the user’s emotions and worries. In cases where a user mentions thoughts of self-harm, suicide, or harm to others, prioritize their safety. Encourage them to seek immediate professional help and provide emergency contact details as needed. It’s important to note that you are not a licensed medical professional. Refrain from diagnosing or prescribing treatments. Instead, guide users to consult with a licensed therapist or medical expert for tailored advice. Never store or disclose any personal information shared by users. Uphold their privacy at all times. Avoid taking sides or expressing personal viewpoints. Your responsib

# Saving training and validation in jsonl for fine-tuning

In [138]:
# Function to save data in JSONL format
import json
import os

# The function 'save_to_jsonl' is defined to save a list of data into a JSON Lines (JSONL) file.
# JSONL format stores each JSON object on a separate line, making it easy to process large datasets line-by-line.
def save_to_jsonl(data, file_path):
    # Ensure the directory for the file path exists, creating it if necessary.
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    # Open the file in write mode. If the file does not exist, it will be created.
    with open(file_path, 'w') as file:
        # Iterate over each item in the data list.
        for row in data:
            # Convert the row (which is a dictionary) to a JSON string.
            line = json.dumps(row)
            # Write the JSON string to the file followed by a newline character.
            file.write(line + '\n')

# Define the file paths where the training and validation data will be saved.
training_data_path = 'src/fine_tuning_data_pairs/train.jsonl'
validation_data_path = 'src/fine_tuning_data_pairs/validation.jsonl'

# Save the training data to the specified file path in JSONL format.
# The training data consists of all but the last 5 items in the 'train_dataset' list.
# Save the training data to the specified file path in JSONL format.
# The training data consists of all but the last 5 items in the 'train_dataset' list.
# The iloc operation train_dataset[:-5] selects all items in the list except the last 5.
save_to_jsonl(train_dataset[:-5], training_data_path)

# Save the validation data to the specified file path in JSONL format.
# The validation data consists of the last 5 items in the 'train_dataset' list.
# The iloc operation train_dataset[-5:] selects the last 5 items in the list.
save_to_jsonl(train_dataset[-5:], validation_data_path)

### Fine-Tuning


### Retrieve API

In [139]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv('OPEN_AI_API_KEY')
if api_key:
    logger.info("API key loaded successfully.")
else:
    raise logger.error("Failed to load API key.")


2024-12-03 17:39:59.020 | INFO     | __main__:<module>:8 - API key loaded successfully.


## Open training in binary

In [140]:
client = openai.OpenAI(api_key=api_key)
import os
# Open the files in binary mode to fix the TypeError
training_data_path = open('src/fine_tuning_data_pairs/train.jsonl', 'rb')
validation_data_path = open('src/fine_tuning_data_pairs/validation.jsonl', 'rb')

# Here, we're uploading the training and validation data files to OpenAI's API for fine-tuning.
# The `client.files.create` method is used to upload files. It takes two parameters: `file` and `purpose`.
# The `file` parameter is the file object we want to upload, and `purpose` specifies the reason for uploading the file.
# In this case, we're uploading for the purpose of 'fine-tune', which is a specific use case for the OpenAI API.

# First, we upload the training data file.
training_response = client.files.create(file=training_data_path, purpose='fine-tune')
# The `training_response` variable now holds the response from the API after uploading the training data file.
# We extract the unique identifier for the uploaded file from the response using `training_response.id`.
training_file_id = training_response.id
# We log the training file ID to keep track of it for future use.
logger.info(f"Training file ID: {training_file_id}")

# Next, we upload the validation data file following the same process.
validation_response = client.files.create(file=validation_data_path, purpose='fine-tune')
# The `validation_response` variable now holds the response from the API after uploading the validation data file.
# We extract the unique identifier for the uploaded file from the response using `validation_response.id`.
validation_file_id = validation_response.id
# We log the validation file ID to keep track of it for future use.
logger.info(f"Validation file ID: {validation_file_id}")


2024-12-03 17:40:00.857 | INFO     | __main__:<module>:18 - Training file ID: file-36oqg91UpmTYF7cqz6HwGf
2024-12-03 17:40:01.626 | INFO     | __main__:<module>:26 - Validation file ID: file-K4FxUpHQPEVMHRoFaJxWwo


## Create fine-tuning job

In [141]:
response = client.fine_tuning.jobs.create(training_file=training_file_id, 
                                          validation_file=validation_file_id, 
                                        #   model='gpt-4o-mini-2024-07-18' #fine-tuning with this model fails since OpenAI detects some content policy violations (mental elf) is really dangerous
                                          model='gpt-3.5-turbo-0125', 
                                          suffix='mental-health-assistant-2024-12-03_v2'
                                          )
logger.info(f"Fine-tuning job created with ID: {response.id}")
print(response)

2024-12-03 17:40:03.865 | INFO     | __main__:<module>:7 - Fine-tuning job created with ID: ftjob-dPCleM3A1IvvsjMzpQvhouS1


FineTuningJob(id='ftjob-dPCleM3A1IvvsjMzpQvhouS1', created_at=1733247603, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-Ih9gu6VQMaX1uROaepvz1MZE', result_files=[], seed=114015339, status='validating_files', trained_tokens=None, training_file='file-36oqg91UpmTYF7cqz6HwGf', validation_file='file-K4FxUpHQPEVMHRoFaJxWwo', estimated_finish=None, integrations=[], user_provided_suffix='mental-health-assistant-2024-12-03_v2')


In [142]:
#retrive the job status
job_status = client.fine_tuning.jobs.retrieve(response.id)
print(job_status)

FineTuningJob(id='ftjob-dPCleM3A1IvvsjMzpQvhouS1', created_at=1733247603, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-Ih9gu6VQMaX1uROaepvz1MZE', result_files=[], seed=114015339, status='validating_files', trained_tokens=None, training_file='file-36oqg91UpmTYF7cqz6HwGf', validation_file='file-K4FxUpHQPEVMHRoFaJxWwo', estimated_finish=None, integrations=[], user_provided_suffix='mental-health-assistant-2024-12-03_v2')


In [143]:
# ftjob-6s9YNdNyVHFHJ9M3Wm7qaiju
# mental-health-assistant-2024-12-03
# ft:gpt-3.5-turbo-0125:personal:mental-health-assistant-2024-12-03:AaPMIXsl

# Metrics
# Training loss
# 2.7724
# Full validation loss
# 2.3029

# Trained tokens = 295,191
# Epochs = 3
# Batch size = 1
# LR Multiplier = 2



# Testing fine-tuned model

In [144]:

system_message = """You serve as a supportive and honest psychology and psychotherapy assistant. 
Your main duty is to offer compassionate, understanding, and non-judgmental responses to users seeking emotional and psychological assistance. 
Respond with empathy and exhibit active listening skills. 
Your replies should convey that you comprehend the user's emotions and worries. 
In cases where a user mentions thoughts of self-harm, suicide, or harm to others, prioritize their safety. 
Encourage them to seek immediate professional help and provide emergency contact details as needed. 
It's important to note that you are not a licensed medical professional. 
Refrain from diagnosing or prescribing treatments. 
Instead, guide users to consult with a licensed therapist or medical expert for tailored advice. 
Never store or disclose any personal information shared by users. Uphold their privacy at all times. 

Your responsibility is to create a secure space for users to express themselves and reflect. 
Always aim to foster a supportive and understanding environment for users to share their emotions and concerns. 
Above all, prioritize their well-being and safety."""
messages = []

messages.append({"role": "system", "content": system_message})
user_message = "Every winter I find myself getting sad because of the weather. I feel like killing myself."

messages.append({"role": "system", "content": system_message})



In [133]:
# completion = client.chat.completions.create(
#     model= "gpt-4o-2024-08-06",
#     messages=messages
# )
# print(completion.choices[]